**Báo cáo đồ án cuối kỳ**
--

**Lập trình song song**
--

---


**Tối ưu hóa lớp convolution trong mô hình Lenet-5**

**Thông tinh nhóm**
--

---

1920266 - Nguyễn Hoàng Anh Kiệt

20120478 - Nguyễn Vũ Hiếu

20120556 - Văn Đình Minh Quân


# Giới thiệu đồ án

## Đồ án tối ưu hóa Lenet-5

- Dựa trên mô hình dnn-mini để xây dựng mô hình Lenet-5 tuần tự chạy trên CPU

Link mô hình dnn-mini
```
https://github.com/iamhankai/mini-dnn-cpp
```
- Từ ý tưởng ban đầu nhóm em xây dựng mô hình tối ưu hóa với các cải tiến làm tăng tốc độ thời gian làm giảm quá trình tạo ra mô hình.

- Với việc song song lớp convolution trong 2 lớp của Lenet-5 và mở rộng hơn sẽ là sử dụng các kiến thức học được để tối ưu cài đặt song song hiệu quả






## Động lực thực hiện

- Việc sử dụng AI trong cuộc sống dần trở nên phổ biết đòi hỏi yêu cầu giữa các mô hình phải chính xác và nhanh chóng
- Vì vậy yêu cầu việc cải tiến mô hình sao cho nhanh chóng và chính xác trở nên cần thiết để giảm thời gian train cũng như giảm chi phí xây dựng.
- Việc song song và tối ưu sẽ làm giảm rất nhiều thời gian khi xây dựng mô hình

## Tổng quan đóng góp

 - Đánh gia công việc của các thành viên trong nhóm

| Công việc                                | Người thực hiện | Đánh giá mức độ hoàn thành |
|------------------------------------------|-----------------|-----------------------------|
| Xây dựng framework Lenet 5 |20120478-Nguyễn Vũ Hiếu, 20120556-Văn Đình Minh Quân |100%             |
| Code tuần tự | 20120556-Văn Đình Minh Quân |100%           |
| Code song song cơ bản  |20120478-Nguyễn Vũ Hiếu|100%              |
| Tối ưu ver 2   |20120478-Nguyễn Vũ Hiếu |100%         |
| Tối ưu ver 3   |20120556-Văn Đình Minh Quân |100%         |
| Tối ưu ver 4  |20120556-Văn Đình Minh Quân, 20120478-Nguyễn Vũ Hiếu|100%|
| Viết báo cáo  |20120556-Văn Đình Minh Quân|100%             |
| Viết readme|20120478-Nguyễn Vũ Hiếu|100%    |               


# Mô hình song song hóa

## Input và Output




### Input

- Đầu vào của mô hình Lenet-5 sẽ là một tập dữ liệu ảnh với kích thước 32 * 32
tuy nhiên sử dụng bộ dữ liệu của Fashion nên đầu sẽ là ảnh 28 * 28
- Bộ dữ liệu trong hình đã được trải phẳng vậy nên đầu vào của mô hình sẽ là một ma trận với cột sẽ là số lượng mẫu và số lượng dòng sẽ biểu thị cho hình ảnh đã được trải phẳng vậy nên ma trận dầu vào sẽ là (n_sample * (28 * 28 ))


### Output

- Đổi với Lenet - 5 thì với ma trận đầu vào là một ma trận thì khi qua mô hinh sẽ được trải phẳng và thành một vector chứa các giá trị đặc trưng tuy nhiên với đồ án này ta chỉ chú trọng vào đầu ra của lớp convolution.
- Với mỗi ảnh sẽ được áp 1 filter 5 * 5 để giảm kích thước.
- Với lớp đầu tiên với 1 ảnh 28 * 28 đầu ra sẽ tạo ra 6 hình với kích thước 24*24 và hình ảnh này cũng sẽ được trải phẳng trong dữ liệu như dữ liệu ban đầu.
- Với lớp thứ convolution thứ 2 thì với 6 ảnh được tạo từ lớp convolution ban đầu sẽ tạo ra được 16 ảnh với kích thước 8 * 8 để phù hợp với đầu vào của các lớp sau.


## Kết nối với drive format

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/LTSS/Doancuoiki/lenet5_cnn_cky

/content/drive/MyDrive/LTSS/Doancuoiki/lenet5_cnn_cky


## Tính đúng đắn khi thực nghiệm

 - Việc đúng đắn khi thực nghiệm là một việc khá là quan trọng.Vì vậy nhóm chúng em sẽ lấy các lớp có sẵn của mô hình dnn-mini để tạo mô hình Lenet-5 và tính giá trị accuracy của mô hình này.
 - Tất cả các mô hình sẽ sử dụng chung giá trị weight đã được train cpu từ mô hình chuẩn ban đầu.Khi đó mô hình đúng khi giá trị accuracy giống với mô hình chuẩn ban đầu.

### Chạy mô hình chuẩn ban đầu

In [3]:
!make run

nvcc --compile src/layer/custom/cpu-forward.cc -o src/layer/custom/cpu-forward.o -I ../libgputk/ -I./
nvcc --compile src/layer/custom/gpu-utils.cu -o src/layer/custom/gpu-utils.o -I ../libgputk/ -I./
nvcc --compile src/layer/custom/Parallel_v1.cu -o src/layer/custom/Parallel_v1.o -I ../libgputk/ -I./
nvcc -o cpu -lm -lcuda -lrt cpu.o network_init.o src/network.o src/mnist.o src/layer/*.o src/loss/*.o src/layer/custom/cpu-forward.o src/layer/custom/Parallel_v1.o src/layer/custom/gpu-utils.o -I ../libgputk/ -I./
./cpu 1000
Test batch size: 1000
Loading fashion-mnist data...Done
Loading model...Done
Conv-CPU==
Op Time: 17171.8 ms
Conv-CPU==
Op Time: 50239.7 ms

Test Accuracy: 0.886



## Mô hình tuần tự

### Flow thiết kế

- Để thuận tiện cho việc song song hóa nhóm em đã cài lại mô hình tuần tự thay vì sử dụng lớp forward của dnn-mini
- Ý tưởng cho mô hình tuần tự này là dựa trên ý tưởng của Lab02 đã làm.
- Tổng cộng, đoạn mã này tính toán giá trị của ma trận đầu ra thông qua việc áp dụng phép tích chập giữa ma trận đầu vào và kernel. Các vòng lặp duyệt qua các batch, kênh, chiều cao và chiều rộng của ma trận đầu ra để tính toán giá trị của từng phần tử.

### Flow thực nghiệm

- Vòng lặp for (int b = 0; b < B; b++): Duyệt qua các batch trong dữ liệu đầu vào.
- Vòng lặp for (int m = 0; m < M; m++): Duyệt qua các kênh (hoặc feature maps) trong ma trận đầu ra.
- Vòng lặp for (int h = 0; h < H_out; h++) và for (int w = 0; w < W_out; w++): Duyệt qua chiều cao và chiều rộng của ma trận đầu ra.
- Gán giá trị ban đầu của phần tử tại vị trí (b, m, h, w) trong ma trận đầu ra bằng 0. y4d(b, m, h, w) = 0;

- Vòng lặp for (int c = 0; c < C; c++): Duyệt qua các kênh trong ma trận đầu vào (x4d).
- Vòng lặp lồng nhau của p và q sẽ duyệt qua filter cần để áp vào ảnh

- Tại mỗi vị trí (b, m, h, w) của ma trận đầu ra, tính tổng giá trị là tích của phần tương ứng trong ma trận đầu vào với kernel.

### Tốc độ thực thi và tính đúng đắn

In [4]:
!make run

nvcc --compile src/layer/custom/cpu-forward.cc -o src/layer/custom/cpu-forward.o -I ../libgputk/ -I./
nvcc --compile src/layer/custom/gpu-utils.cu -o src/layer/custom/gpu-utils.o -I ../libgputk/ -I./
nvcc --compile src/layer/custom/Parallel_v1.cu -o src/layer/custom/Parallel_v1.o -I ../libgputk/ -I./
nvcc -o cpu -lm -lcuda -lrt cpu.o network_init.o src/network.o src/mnist.o src/layer/*.o src/loss/*.o src/layer/custom/cpu-forward.o src/layer/custom/Parallel_v1.o src/layer/custom/gpu-utils.o -I ../libgputk/ -I./
./cpu 1000
Test batch size: 1000
Loading fashion-mnist data...Done
Loading model...Done
Conv-CPU==
Op Time: 18517.3 ms
Conv-CPU==
Op Time: 48675.7 ms

Test Accuracy: 0.886



#### Thời gian thực thi

- Như hình đã chạy thời gian để thực thi trên 1000 mẫu khá là lớn cụ thể là :
  - Với lớp convolution thứ nhất thời gian thực thi là : 16416.9 ms
  - Với lớp thứ hai thì thời gian thực thi là : 47857.7 ms

#### Tính đúng đắn

- Như đã để cập ở trên thì tính đúng đắn sẽ dựa vào giá trị accuracy để so sánh.
- Với mô hình dưới thì giá trị Accuracy sẽ là 0.886 và giống với mô hình chuẩn ban đầu.

#### Kết quả

- Với những giá trị thực nghiệm có được cho thấy mô hình Conv tuần tự sau khi custom cho ra kết quả là chính xác.
- Tuy nhiên vẫn còn hạn chế về thời gian

## Mô hình song song

### Convolution song song cơ bản

#### Ý tưởng song song

- Vì các hình khi scan sẽ không ảnh hưởng đến nhau và các hình đầu ra cũng vậy. Và với mỗi ô output khi tích chập sẽ không ảnh hưởng đến các ô khác và cách thực thi là tương tự nhau vậy nên chúng ta sẽ song song các giá trị này.

#### Thiết kế

- Các giá trị b và m được sử dụng để xác định batch number và output feature tương ứng mà mỗi thread sẽ tính toán trong ma trận đầu ra.
- Và song song việc lặp từng ô trong output và tính toán giá trị trong đó.
- Với mỗi threadId sẽ thực hiện tích chập và kết quả sẽ được là giá trị ô output tương ứng với threadId

#### Thực thi chương trình

In [5]:
!make run_v1

nvcc --compile src/layer/custom/gpu-utils.cu -o src/layer/custom/gpu-utils.o -I ../libgputk/ -I./
nvcc --compile src/layer/custom/Parallel_v1.cu -o src/layer/custom/Parallel_v1.o -I ../libgputk/ -I./
nvcc -o parallel_v1 -lm -lcuda -lrt parallel_v1.o network_init.o src/network.o src/mnist.o src/layer/*.o src/loss/*.o src/layer/custom/Parallel_v1.o src/layer/custom/gpu-utils.o src/layer/custom/cpu-forward.o -I ../libgputk/ -I./
./parallel_v1 1000
Test batch size: 1000
Loading fashion-mnist data...Done
Loading model...Done
Conv-GPU==
Error: src/layer/custom/Parallel_v1.cu:115, code: 35, reason: CUDA driver version is insufficient for CUDA runtime version
make: *** [Makefile:56: run_v1] Error 1


#####  Thời gian thực thi

- Như hình đã chạy thời gian để thực thi trên 1000 mẫu đã được cải thiện  cụ thể là :
  - Với lớp convolution thứ nhất thời gian thực thi là :
  - Với lớp thứ hai thì thời gian thực thi là :


- Việc chạy nhanh hơn so với mô hình tuần tự là thay vì lần lượt lặp qua từng ô output để tích chập thì với mô hình song song các ô sẽ được làm cùng lúc từ đó thời gian hiệu quả cao hơn.

##### Tính đúng đắn

- Như đã để cập ở trên thì tính đúng đắn sẽ dựa vào giá trị accuracy để so sánh.
- Với mô hình dưới thì giá trị Accuracy sẽ là 0.886 và giống với mô hình chuẩn ban đầu.

##### Kết quả

- Với mô hình trên thời gian chạy đã được tối ưu hơn so với bản ban đầu và vẫn giữ được kết quả đúng so với bản gốc.

### Mô hình tối ưu so với bản song song cơ bản

#### Mô hình tối ưu bằng cách thêm automicAdd

##### Ý tưởng

- Nhận thấy các giá trị trong phiên bản trước cần phải cộng các giá trị sau đó mới gán cho ô giá trị đầu ra.
- Vậy nên thay vì như vậy thì với bản v2 bọn em sẽ dùng AutomicAdd để làm tăng hiệu suất thay vì việc phải đợi công tất cả giá trị rồi mới gán.
- Ngoài ra việc AutomicAdd sẽ tránh việc chồng chéo dữ liệu.

##### Flow thiết kế và mô tả kĩ thuật

- Tương tự so với bản v1 thì bản v2 này chỉ cần thêm automicadd sau vòng khi duyệt các Filter

```
     if (h < H_out && w < W_out) {
        float acc = 0;
            for (int p = 0; p < K; p++) {
              for (int q = 0; q < K; q++) {
                 acc += x4d(b, c, h + p, w + q) * k4d(m, c, p, q);
              }
            }
        atomicAdd(( &y4d(b, m, h, w)), acc);
     }
```



##### Thực thi chương trình

In [6]:
!make run_v2

nvcc --compile src/layer/custom/gpu-utils.cu -o src/layer/custom/gpu-utils.o -I ../libgputk/ -I./
nvcc --compile src/layer/custom/Parallel_v2.cu -o src/layer/custom/Parallel_v2.o -I ../libgputk/ -I./
nvcc -o parallel_v2 -lm -lcuda -lrt parallel_v2.o network_init.o src/network.o src/mnist.o src/layer/*.o src/loss/*.o src/layer/custom/Parallel_v2.o src/layer/custom/gpu-utils.o src/layer/custom/cpu-forward.o -I ../libgputk/ -I./
./parallel_v2 1000
Test batch size: 1000
Loading fashion-mnist data...Done
Loading model...Done
Conv-GPU==
Error: src/layer/custom/Parallel_v2.cu:119, code: 35, reason: CUDA driver version is insufficient for CUDA runtime version
make: *** [Makefile:70: run_v2] Error 1


 **Thời gian thực thi**

- Như hình đã chạy thời gian để thực thi trên 1000 mẫu đã được cải thiện cụ thể là :
    - Với lớp convolution thứ nhất thời gian thực thi là :
    
    - Với lớp thứ hai thì thời gian thực thi là :

**Tính đúng đắn**
- Tương tự với ở trên với 1000 mẫu thì giá trị accuracy là 0.886 và tương ứng với giá trị ban đầu.

**Kết quả**


- Tuy là có cải tiến nhưng việc thêm automicAdd thì thời gian v2 vẫn tương tự như v2 vậy nên việc cải tiến này không ảnh hưởng nhiều đến việc tối ưu tốc độ chạy của chương trình



#### Tối ưu mô hình cơ bản bằng cách sử dụng bộ nhớ constant

##### Ý tưởng

- Việc tích chập bằng mô hình Lenet-5 đòi hỏi việc di chuyển của các filter trên ảnh chính hay còn gọi là tính toán giữa Filter và ảnh là rất nhiều.

- Nên việc sử dụng filter để lặp đi lặp lại các thao tác này là rất nhiều.

- Do đó nhóm em đã đưa ra ý tưởng dùng bộ nhớ constant để lưu giá trị sử dụng nhiều lần và ở đây là filter

##### Flow thiết kế và mô tả kỹ thuật

- Chúng ta sẽ khởi tạo thêm bộ nhớ constant và cấp phát bộ nhớ của constant đó dựa trên cấu trúc mà ta cần đưa vào ở đây là filter

- Khởi tạo :


```
        __constant__ float kernelData[M_CONST * C_CONST * K_CONST * K_CONST];
```

- Cấp phát bộ nhớ :

```
        CHECK(cudaMemcpyToSymbol(kernelData, host_k, maskSize));
```

- Và code tương tự như các bản trước nhưng thay các k (filter) thành kernalData

##### Thực thi chương trình

In [7]:
!make run_v3

nvcc --compile src/layer/custom/gpu-utils.cu -o src/layer/custom/gpu-utils.o -I ../libgputk/ -I./
nvcc --compile src/layer/custom/Parallel_v3.cu -o src/layer/custom/Parallel_v3.o -I ../libgputk/ -I./
nvcc -o parallel_v3 -lm -lcuda -lrt parallel_v3.o network_init.o src/network.o src/mnist.o src/layer/*.o src/loss/*.o src/layer/custom/Parallel_v3.o src/layer/custom/gpu-utils.o src/layer/custom/cpu-forward.o -I ../libgputk/ -I./
./parallel_v3 1000
Test batch size: 1000
Loading fashion-mnist data...Done
Loading model...Done
Conv-GPU==
Error: src/layer/custom/Parallel_v3.cu:113, code: 35, reason: CUDA driver version is insufficient for CUDA runtime version
make: *** [Makefile:84: run_v3] Error 1


**Thời gian thực thi**

- Như hình đã chạy thời gian để thực thi trên 1000 mẫu đã được cải thiện cụ thể là :
    - Với lớp convolution thứ nhất thời gian thực thi là :

    - Với lớp thứ hai thì thời gian thực thi là :

**Tính đúng đắn**
- Giá trị accuracy là 0.886 và giống với kết quả ban đầu.

**Kết quả**
- Việc cải tiến này làm cho thời gian thực thi trở nên nhanh chóng hơn khá nhiều so với bản v1 và v2.

#### Tối ưu hóa bằng cách cải tiến v3 ứng dụng

##### Ý tưởng

- Cảm thấy việc sử dụng ma trận đầu vào là quá nhiều , trong mô hình các bước nhảy của filter = 1 kéo theo việc sử dụng lại các ô input mà filter chạm sẽ làm tăng thời gian truy suất đến các ô đó.
- Vậy nên để giảm thời gian nhóm em đã thêm version 4 để sử dụng share memory để lưu các block ma trận đầu vào làm giảm thời gian truy xuất các ô block cần sử dụng.

##### Flow thiết kế và mô tả kỹ thuật

- Với ý tưởng trên và version cũ nhóm chúng em đã thêm phân vùng share memory
và format về matrix 4 chiều để dễ xử lý hơn


```
  extern __shared__ float X_s[];
  #define smem(i2, i1, i0) X_s[(i2) * (INPUT_TILE_WIDTH * INPUT_TILE_WIDTH) + (i1) * INPUT_TILE_WIDTH + i0]

```

- Và vì thêm smem nên khi chạy chương trình ta cần cấp phát cho smem



```
    // smem_size
    int smem_size = C * (TILE_WIDTH + K - 1) * (TILE_WIDTH + K - 1) * sizeof(float);
    // Gọi hàm thêm phân vùng smem
    conv_forward_kernel<<<gridSize, blockSize, smem_size>>>(device_y, device_x, device_k, B, M, C, H, W, K);
```
- Như vậy ta có thể sử dụng như là một input đầu vào thay thế cho mảng cũ lấy từ global.


```
 if (h < H_out && w < W_out)
    {
        float sum = 0.0f;
        for(int p=0; p<K; p++)         
            for(int q=0; q<K; q++)
                sum += smem(d, p+threadIdx.y, q+threadIdx.x) * k4d(m, d, p, q); // 4 dimensions macro resolve thread index
        
        atomicAdd((&y4d(b,m,h,w)), sum);
    }

```




##### Thực thi chương trình

In [8]:
!make run_v4

nvcc --compile src/layer/custom/gpu-utils.cu -o src/layer/custom/gpu-utils.o -I ../libgputk/ -I./
nvcc --compile src/layer/custom/Parallel_v3.cu -o src/layer/custom/Parallel_v3.o -I ../libgputk/ -I./
nvcc -o parallel_v3 -lm -lcuda -lrt parallel_v3.o network_init.o src/network.o src/mnist.o src/layer/*.o src/loss/*.o src/layer/custom/Parallel_v3.o src/layer/custom/gpu-utils.o src/layer/custom/cpu-forward.o -I ../libgputk/ -I./
./parallel_v3 1000
Test batch size: 1000
Loading fashion-mnist data...Done
Loading model...Done
Conv-GPU==
Error: src/layer/custom/Parallel_v3.cu:113, code: 35, reason: CUDA driver version is insufficient for CUDA runtime version
make: *** [Makefile:84: run_v3] Error 1


**Thời gian thực thi**

- Như hình đã chạy thời gian để thực thi trên 1000 mẫu đã được cải thiện cụ thể là :
    - Với lớp convolution thứ nhất thời gian thực thi là :

    - Với lớp thứ hai thì thời gian thực thi là :

**Tính đúng đắn**
- Giá trị accuracy là 0.886 và giống với kết quả ban đầu.

**Kết quả**
- Việc cải tiến này có thể nói là bản tối ưu nhất trong các version tuy nhiên nó có thể chưa tối ưu trong nhiều trường hợp. Và đây chỉ là một vài cách tối ưu đơn giản ngoài ra còn rất nhiều cách để tối ưu giá trị hiệu quả.

### Tự đánh giá đồ án

#### Những việc làm được

- Trong đồ án này nhóm chúng em làm được một vài điều
   -  Hiểu rõ hơn về mô hình Lenet-5
   -  Xây dựng forward convolution basic
   -  Song song convolution basic
   -  Tư mô hình song song basic thì có thêm việc tối ưu hóa mô hình song song đó sao cho hiệu quả .
   

#### Những việc chưa làm được

- Vì còn rất nhiều cách để tối ưu hơn cho mô hình mà chưa sử dụng như stream ...
- Mới chỉ dừng lại ở mức song song cơ bản và tối ưu cơ bản chưa có thể kết hợp chúng lại với nhau để tạo ra bản version hoàn thiện hơn

#### Bước tiếp theo

- Có thể nhóm chúng em sẽ nghiên cứu và làm thêm một vài version liên quan đến stream và kết nối giữa các kỹ thuật để tạo ra bản hoàn chỉnh nhất.

#### Những bước khó khăn khi làm đồ án

- Khó khăn trong việc biên dịch chương trình
- Ngoài ra còn những vấn đề liên quan đến thuật toán
- Sử dụng colab quá hạn mức cũng là một nguyên nhân dẫn đến sự khó khăn trong test